In [6]:
import numpy as np
import tensorflow as tf

def normal_sampling(mu1, v1, mu2, v2, nrow):
    x = np.random.normal(mu1, v1, nrow)
    y = np.random.normal(mu2, v2, nrow)
    return np.vstack([x,y])
"""
다양한 평균과 분산에 대해서 샘플링하여 쌓아줍니다. 
np.hstack은 가로로 붙여줍니다. 
그래서 마지막에 Transpose 했습니다. 
"""
sample_size = 500
cluster_num = 3
x_data = np.hstack([
    normal_sampling(0, 1, 0, 1, sample_size),
    normal_sampling(2, 1, 2, 1, sample_size), 
    normal_sampling(3, 1, 7, 1, sample_size),
    normal_sampling(8, 1, 4, 1, sample_size),
    normal_sampling(6, 1, 5, 1, sample_size),
    normal_sampling(6, 3, 0, 1, sample_size),
    normal_sampling(0, 3, 6, 2, sample_size)
]).T
y_data = []
for i in range(0, x_data.shape[0]//sample_size):
    y_data+=[i for j in range(0, sample_size)]
y_data = np.array(y_data)

In [45]:
x_data.shape

(3500, 2)

In [51]:
nb_features = 2
nb_classes = 7

X = tf.placeholder(tf.float32, [None, nb_features])
Y = tf.placeholder(tf.int32, [None])  # 0 ~ 6
Y_one_hot = tf.one_hot(Y, nb_classes)

W = tf.Variable(tf.random_normal([nb_features, nb_classes]), name='weight')
b = tf.Variable(tf.random_normal([nb_classes]), name='bias')
"""
logit으로 계산한 각 class 에 대한 값을 softmax function에 넘겨서 계산해줌
- exponential normalization? 같은 느낌이라고 생각하면 됨
"""
logits = tf.matmul(X, W) + b
hypothesis = tf.nn.softmax(logits)

# Cross entropy cost/loss
cost_i = tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=Y_one_hot)
cost = tf.reduce_mean(cost_i)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.1).minimize(cost)

"""
- tf.argmax를 하면, axis를 축으로 변화하며 가장 큰 것의 index들을 리턴함
"""
prediction = tf.argmax(hypothesis, 1)
correct_prediction = tf.equal(prediction, tf.argmax(Y_one_hot, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
# Launch graph
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    
    for step in range(5000):
        sess.run(optimizer, feed_dict={X: x_data, Y: y_data})
        if step % 300 == 0:
            loss, acc = sess.run([cost, accuracy], feed_dict={X: x_data, Y: y_data})
            print("Step: {:5}\tLoss: {:.3f}\tAcc: {:.2%}".format(step, loss, acc))
    # Let's see if we can predict
    pred = sess.run(prediction, feed_dict={X: x_data})
    # y_data: (N,1) = flatten => (N, ) matches pred.shape
    """
    for p, y in zip(pred, y_data.flatten()):
        if p != int(y):
            print("[{}] Prediction: {} True Y: {}".format(p == int(y), p, int(y)))
    """


Step:     0	Loss: 7.286	Acc: 13.51%
Step:   300	Loss: 0.750	Acc: 79.66%
Step:   600	Loss: 0.669	Acc: 81.26%
Step:   900	Loss: 0.634	Acc: 81.49%
Step:  1200	Loss: 0.614	Acc: 81.74%
Step:  1500	Loss: 0.602	Acc: 81.89%
Step:  1800	Loss: 0.592	Acc: 82.14%
Step:  2100	Loss: 0.586	Acc: 82.14%
Step:  2400	Loss: 0.580	Acc: 82.09%
Step:  2700	Loss: 0.576	Acc: 82.09%
Step:  3000	Loss: 0.573	Acc: 82.20%
Step:  3300	Loss: 0.570	Acc: 82.26%
Step:  3600	Loss: 0.567	Acc: 82.34%
Step:  3900	Loss: 0.565	Acc: 82.20%
Step:  4200	Loss: 0.563	Acc: 82.09%
Step:  4500	Loss: 0.561	Acc: 82.17%
Step:  4800	Loss: 0.560	Acc: 82.14%


In [37]:
"""axis 복습: 입력받은 axis만 변화하고, 
"""
sess = tf.Session()
a = np.array([1,2,3,4,5,6]).reshape(2, 3)
print(a.shape)
print(a)
sess.run(tf.argmax(a, axis=0))# 값을 리턴하는 것이 아니라, 값의 위치를 리턴함

(2, 3)
[[1 2 3]
 [4 5 6]]


array([1, 1, 1])

In [41]:
np.min(a, axis=0)

array([1, 2, 3])